# Neural Networks retrieval on the Satellite Observations Dataset

In this final step we develop a model to estimate soil moisture using neural networks

Libraries we need are imported first

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import xarray as xr
from scipy.interpolate import griddata
from scipy import interpolate
import statsmodels.api as sm

Data used in this exercise is stored in the DATA folder

In [5]:
thefile = '../../../DATA/TPdata_199301'
data = pd.read_csv(thefile, sep='\t', header=None)
data.columns = ["cellNr", "latitude", "longitude", "backscatter","emissivity_v","emissivity_h","ts_amplitude","ndvi","lmd_soilWetness"]
data = data.replace('   NaN',pd.NA)
data['backscatter'] = pd.to_numeric(data['backscatter'])
data['emissivity_v'] = pd.to_numeric(data['emissivity_v'])
data['emissivity_h'] = pd.to_numeric(data['emissivity_h'])
data['ts_amplitude'] = pd.to_numeric(data['ts_amplitude'])
data['ndvi'] = pd.to_numeric(data['ndvi'])
data['lmd_soilWetness'] = pd.to_numeric(data['lmd_soilWetness'])
data = data.dropna()

## Neural Network

In [7]:
data

,cellNr,latitude,longitude,backscatter,emissivity_v,emissivity_h,ts_amplitude,ndvi,lmd_soilWetness
63715,63716.0,-53.875,-71.0247,-11.7482,0.8083,0.6605,22.8432,0.4072,0.4771
65425,65426.0,-53.375,-71.4552,-11.6666,0.8497,0.7343,24.4724,0.1540,0.5946
68907,68908.0,-52.375,-71.4676,-15.0033,0.9559,0.9072,36.5638,0.4047,0.5316
68908,68909.0,-52.375,-71.0580,-16.3849,0.9428,0.8842,39.3991,0.4233,0.4172
70678,70679.0,-51.875,-71.4736,-17.0347,0.9955,0.9738,40.4082,0.2735,0.4525
...,...,...,...,...,...,...,...,...,...
596855,596856.0,53.875,-75.6890,-11.6752,0.9700,0.8773,9.6753,0.0110,0.6116
597727,597728.0,54.125,-64.1943,-11.1028,0.9802,0.8938,5.9767,0.0020,0.8028
601035,601036.0,55.125,-73.2685,-12.4459,1.0079,0.9262,12.8538,-0.0140,0.8286
604434,604435.0,56.125,-4.7073,-9.9505,0.9586,0.9155,5.0433,0.1880,0.9225


## Performance of the Neural Network